# **Building ReAct Agents**

## **What's Covered?**
1. Introduction to Agent
    - What is an Agent?
    - What is a ReAct Agent?
    - Understanding the ReAct Loop
    - What is Langchain's create_agent()?
    - Implementing an Agent with Core Component - Model, Tools and Prompt
    - A simple implementation - Input Output
2. Implementing an Agent
    - Agent's Core Component - Model, Tools and Prompt
    - Step 1: Init Chat Models
    - Step 2: Init an Agent
    - Step 3: Invoking the Agent
    - Step 4: Understanding Models Response
    - Step 5: Pretty Printing the Model's Response
3. Enforcing Structured Output
    - Introduction to Structured Output
    - Implementing Structured Responses
    - Step 1: Defining the Schema for response_format
    - Step 2: Agent Init with ProviderStrategy for Structured Output
    - Step 3: Agent Init with Invoking the Agent and Printing structured_response
    - Step 4: ToolStrategy for Structured Output
    - Passing Multiple Schemas
4. Error Handling with ToolStrategy
    - Multiple Structured Outputs Error
    - Schema Validation Error
5. Streaming Responses

Topics to cover:
- Streaming (Real-time UX)
- Built-in Middleware



## **Introduction to Agent**

### **What is an Agent?**
Agents combine language models with tools to create systems that can reason about tasks, decide which tools to use, and iteratively work towards solutions.

An LLM Agent runs tools in a loop to achieve a goal. An agent runs until a stop condition is met - i.e., when the model emits a final output or an iteration limit is reached.

### **What is a ReAct Agent**?
ReAct is an agent pattern where the LLM alternates between Reason (think) and Act (use tools) in a loop until it solves the task.
In short, 
> **Agent = Reasoning and Decision Making (using LLMs) + Action (using Tools)**

### **Understanding the ReAct Loop**
Imagine asking an agent: "What's the weather in SF and book a flight if sunny?"
```
1. Reason: "First check weather. Use get_weather tool."
   ↓ (calls tool)
2. Act: Tool runs → "SF: Sunny 72°F"
   ↓ (feeds back)
3. Reason: "Sunny! Now book flight. Use book_flight tool."
   ↓ (calls tool)
4. Act: Tool runs → "Flight booked: UA123"
   ↓
5. Finish: "Weather sunny. Flight booked."
```

### **What is Langchain's create_agent()?**
In Langchain **create_agent()** provides a production-ready agent implementation.

In create_agent(): Built-in ReAct—model reasons, picks tools, loops automatically.

Introduced in LangChain 1.x, **create_agent()** is a graph-based ReAct agent (built on LangGraph) that follows **model → tools → model** loops until completion.


**create_agent** builds a graph-based agent runtime using LangGraph. A graph consists of nodes (steps) and edges (connections) that define how your agent processes information. The agent moves through this graph, executing nodes like the model node (which calls the model), the tools node (which executes tools), or middleware.

## **Implementing an Agent**

### **Agent's Core Component - Model, Tools and Prompt**

Every agent needs a model, tools (optional), and system_prompt (optional).

**Syntax**
```python
from langchain.agents import create_agent
from langchain.tools import tool

@tool
def check_weather():
    "This check the weather"
    return "Hot"

agent = create_agent(
    model=llm,
    tools=[check_weather],
    system_prompt="You are a helpful assistant with weather tools."  # Shapes behavior
)
```

### **Step 1: Init Chat Models**

In [1]:
# Setup API Key

f = open('keys/.gemini.txt')

GOOGLE_API_KEY = f.read()

In [2]:
# Setup API Key

f = open('keys/.openai_api_key.txt')

OPENAI_API_KEY = f.read()

In [3]:
# Setup API Key

f = open('keys/.groq_api_key.txt')

GROQ_API_KEY = f.read()

In [58]:
# Import ChatModel
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

# Pass the standard parameters during initialization
google_chat_model = ChatGoogleGenerativeAI(api_key=GOOGLE_API_KEY, 
                                           model="gemini-3-flash-preview", 
                                           temperature=1)

openai_chat_model = ChatOpenAI(api_key=OPENAI_API_KEY, 
                               model="gpt-4o-mini", 
                               temperature=1)

groq_chat_model = ChatGroq(api_key=GROQ_API_KEY, 
                           model="openai/gpt-oss-20b", 
                           temperature=1, 
                           )

### **Step 2: Init an Agent**

In [73]:
from langchain.agents import create_agent

agent = create_agent(
    model=openai_chat_model,
    system_prompt="You are a helpful AI assistant"
)

### **Step 3: Invoking the Agent**

In [74]:
result = agent.invoke({ 
    "messages": [{"role": "human", "content": "Hi, My name is ThatAIGuy."}] 
})

### **Step 4: Understanding Models Response**

The Agents response will be a key value pair with following keys:
1. **messages:** Always present in every agent response. **messages** contains the complete conversation history including user inputs, model reasoning, tool calls, and tool results. Each message can be displayed using .pretty_print() for readable formatting.
2. **structured_response:** Parsed and validated output that matches your defined schema. Available only when you specify a **response_format** parameter inside **create_agent()**. Automatically handles validation errors and retries.

In [77]:
print(result.keys())

dict_keys(['messages'])


### **Step 5: Pretty Printing the Model's Response**

In [75]:
for msg in result["messages"]:
    msg.pretty_print()

================================ Human Message =================================

Hi, My name is ThatAIGuy.
================================== Ai Message ==================================

Hello, ThatAIGuy! How can I assist you today?


## **Enforcing Structured Output**

### **Introduction to Structured Output**
Structured output allows agents to return data in a specific, predictable format. Instead of parsing natural language responses, you get structured data in the form of JSON objects, Pydantic models, or dataclasses that your application can directly use. 


### **Implementing Structured Responses** 
We can enforce structured output within **create_agent()** using **response_format** to return validated data in a specific schema (Pydantic/JSON) instead of free-form text.

**response_format** parameter inside **create_agent()** controls how the agent returns structured data. To apply structured output, you should either use ProviderStrategy (default) or ToolStrategy.

- **ProviderStrategy**: Some model providers support structured output natively through their APIs (e.g. OpenAI, Grok, Gemini). This is the most reliable method when available.
- **ToolStrategy**: For models that don’t support native structured output, LangChain uses tool calling to achieve the same result. This works with all models that support tool calling, which is most modern models.


**Syntax**
```python
from langchain.agents.structured_output import ProviderStrategy, ToolStrategy
```

### **Step 1: Defining the Schema for response_format**

In [5]:
from pydantic import BaseModel, Field

class ContactInfo(BaseModel):
    """Contact information for a person."""
    name: str = Field(description="The name of the person")
    email: str = Field(description="The email address of the person")
    phone: str = Field(description="The phone number of the person")

### **Step 2: Agent Init with ProviderStrategy for Structured Output**

Some model providers support structured output natively through their APIs (e.g. OpenAI, Grok, Gemini). This is the most reliable method when available.

```python
class ProviderStrategy(Generic[SchemaT]):
    schema: type[SchemaT]
    strict: bool | None = None
```

- **schema:** The schema defining the structured output format.
- **strict:** Optional boolean parameter to enable strict schema adherence. Supported by some providers (e.g., OpenAI and xAI). Defaults to None (disabled).

LangChain automatically uses ProviderStrategy when you pass a schema type directly to create_agent.response_format and the model supports native structured output:

```python
agent = create_agent(
    model=openai_chat_model,
    response_format=ContactInfo,  # Auto-selects ProviderStrategy
    system_prompt="""You are a helpful assistant. 
                     Be concise and accurate. 
                     You are suppose to extract contact info from the user inputs.
                     """
)
```

In [13]:
from typing import Union
from langchain.agents import create_agent
from langchain.agents.structured_output import ProviderStrategy

agent = create_agent(
    model=openai_chat_model,
    response_format=ProviderStrategy(schema=Union[ContactInfo,]),
    system_prompt="""You are a helpful assistant. 
                     Be concise and accurate. 
                     You are suppose to extract contact info from the user inputs.
                     """
)

### **Step 3: Invoking the Agent and Printing structured_response**

LangChain’s **create_agent** handles structured output automatically. The user sets their desired structured output schema, and when the model generates the structured data, it’s captured, validated, and **returned in the `'structured_response'` key** of the agent’s state.

In [14]:
result = agent.invoke({
    "messages": [{"role": "human", 
                  "content": "My name is John Doe, email john@example.com, and number (555) 123-4567"}]
})

print(result["structured_response"])

name='John Doe' email='john@example.com' phone='(555) 123-4567'


### **Step 4: Agent Init with ToolStrategy for Structured Output**

For models that don’t support native structured output, LangChain uses tool calling to achieve the same result. This works with all models that support tool calling, which is most modern models.

```python
class ToolStrategy(schema):
    schema: schema,
    tool_message_content: str | None,
    handle_errors: ERROR_HANDLING_LOGIC
```

- **schema:** The schema defining the structured output format.
- **tool_message_content:** The tool_message_content parameter allows you to customize the message that appears in the conversation history when structured output is generated as per the given schema.
- **handle_errors:** Error handling strategy for structured output validation failures. By default ERROR_HANDLING_LOGIC is set to boolean value True.

In [10]:
from typing import Union
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy

agent = create_agent(
    model=groq_chat_model,
    response_format=ToolStrategy(schema=Union[ContactInfo,]),  # Default: handle_errors=True
    system_prompt="""You are a helpful assistant. 
                     Be concise and accurate. 
                     You are suppose to extract contact info from the user inputs.
                     """
)

In [11]:
result = agent.invoke({
    "messages": [{"role": "human", "content": "My name is John Doe, email john@example.com, and number (555) 123-4567"}]
})

print(result["structured_response"])

name='John Doe' email='john@example.com' phone='(555) 123-4567'


### **Important: Passing Multiple Schemas** 
1. The `Union[ContactInfo,]` enables multiple possible schemas—the model picks the best match (or you get type errors if invalid).
2. Without Union, only one exact schema allowed. `Union[ContactInfo,]` (note trailing comma) is a single-item Union—technically valid Python but redundant here.
3. Agent auto retries if:
    - Wrong schema picked
    - Validation fails (eg: missing email id)

## **Error Handling**

### **Multiple structured outputs error**

Models can make mistakes when generating structured output via tool calling. 

Sometimes cheap llms (like gpt-4o-mini) can greedily calls 2+ schemas. This is rare with good models/prompts.

LangChain provides intelligent retry mechanisms to handle these errors automatically.

In [19]:
from pydantic import BaseModel, Field

class ContactInfo(BaseModel):
    name: str = Field(description="Person's name")
    email: str = Field(description="Email address")

class EventDetails(BaseModel):
    event_name: str = Field(description="Name of the event")
    date: str = Field(description="Event date")

In [31]:
from typing import Union
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy

agent = create_agent(
    model=openai_chat_model,
    tools=[],
    response_format=ToolStrategy(Union[ContactInfo, EventDetails], handle_errors=True),
)

In [32]:
response = agent.invoke({
    "messages": [{"role": "user", 
                  "content": "Extract info: John Doe (john@email.com) is organizing Tech Conference on March 15th"}]
})

for msg in response["messages"]:
    msg.pretty_print()

================================ Human Message =================================

Extract info: John Doe (john@email.com) is organizing Tech Conference on March 15th
================================== Ai Message ==================================
Tool Calls:
  ContactInfo (call_JwID6b2dDRDk7D7VoDTAYNKd)
 Call ID: call_JwID6b2dDRDk7D7VoDTAYNKd
  Args:
    name: John Doe
    email: john@email.com
  EventDetails (call_RTWqfgRvr8aQINLFyxskznjU)
 Call ID: call_RTWqfgRvr8aQINLFyxskznjU
  Args:
    event_name: Tech Conference
    date: March 15th
================================= Tool Message =================================
Name: ContactInfo

Error: Model incorrectly returned multiple structured responses (ContactInfo, EventDetails) when only one is expected.
 Please fix your mistakes.
================================= Tool Message =================================
Name: EventDetails

Error: Model incorrectly returned multiple structured responses (ContactInfo, EventDetails) when only one i

### **Important Consideration**
**Note:** This can increase the token consumption.

**Idea:** Refactor Schema Handling instead of juggling multiple schemas, split the process into a parallel workflow. Create two distinct nodes, where each node is responsible for a single response_format.

### **Schema validation error**

When structured output doesn’t match the expected schema, the agent provides specific error feedback.

In [34]:
from pydantic import BaseModel, Field

class ProductRating(BaseModel):
    rating: int | None = Field(description="Rating from 1-5", ge=1, le=5)
    comment: str = Field(description="Review comment")

In [71]:
agent = create_agent(
    model=openai_chat_model,
    tools=[],
    response_format=ToolStrategy(ProductRating),  # Default: handle_errors=True
    system_prompt="""You are a helpful assistant that parses product reviews and ratings. 
                     Parse EXACTLY as user says. Do NOT fix values. Even if user gives rating >5.
                     Do not make any false field or false value."""
)

In [72]:
response = agent.invoke({
    "messages": [{"role": "user", "content": "User mentioned Amazing product, 10/10!"}]
})

for msg in response["messages"]:
    msg.pretty_print()

================================ Human Message =================================

User mentioned Amazing product, 10/10!
================================== Ai Message ==================================
Tool Calls:
  ProductRating (call_KUq2BzNUkIsLdb7D4TU17FjG)
 Call ID: call_KUq2BzNUkIsLdb7D4TU17FjG
  Args:
    rating: 10
    comment: Amazing product, 10/10!
================================= Tool Message =================================
Name: ProductRating

Error: Failed to parse structured output for tool 'ProductRating': Failed to parse data to ProductRating: 1 validation error for ProductRating
rating
  Input should be less than or equal to 5 [type=less_than_equal, input_value=10, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/less_than_equal.
 Please fix your mistakes.
================================== Ai Message ==================================
Tool Calls:
  ProductRating (call_W3FNylcvW0JNy8B8oNGZyTFH)
 Call ID: call_W3FNylcvW0JNy8B8oNGZ

### **Important Consideration**

**ISSUE 1:** Some models will intentionally fix 10 to 5 at server-side because they knows your schema constraints (ge=1, le=5) from the description.
> **SOLUTION 1:** Use a strict prompt to enforce model to parse the EXACT values.

**ISSUE 2:** Groq does strict validation server-side. Due to this it will throw 400 error even before LangChain's `handle_error=True` can catch/convert to `ToolMessage` retry. 
> **SOLUTION 2:** We can disable this behaviour by passing `kwargs={"strict": "false"}`.

### **Custom Errors**

**Coming Soon**

## **Streaming Agent's Response**

In [ ]:
for token, metadata in agent.stream(
    {"messages": "Calculate 123456 * 789012"},
    stream_mode="messages"    
):
    if token.content:
        print(token.content, end="", flush=True)

## **Agent Memory**

Agents maintain conversation history automatically through the message state. You can also configure the agent to use a custom state schema to remember additional information during the conversation.

Information stored in the state can be thought of as the **short-term memory** of the agent:

Custom state schemas must extend AgentState as a TypedDict.

There are two ways to define custom state:
- Via middleware (preferred)
- Via state_schema on create_agent

## Step 3: Custom State Schema (Beyond Messages)

**Add fields like `user_preferences` to state for memory/tools.**

```python
import pydantic
state_schema = pydantic.BaseModel(
    messages=list[str],
    user_preferences=dict[str, str]  # Persists across calls
)

agent = create_agent(
    model=model,
    tools=[get_weather],
    state_schema=state_schema
)
```

## **Next**

Use the following reference for the Custom Error Handling.

Reference: https://docs.langchain.com/oss/python/langchain/structured-output#param-schema-1

## Step 4: Built-in Middleware (Prebuilt Superpowers)

**Add via `middleware=[]` list. Stack them.**

```python
from langchain.agents.middleware import (
    summarization_middleware,
    human_in_the_loop_middleware,
    pii_redaction_middleware
)

agent = create_agent(
    model=model,
    tools=[get_weather],
    middleware=[
        summarization_middleware(model="gpt-4o-mini", trigger={"tokens": 500}),  # Auto-summarize history
        human_in_the_loop_middleware(interrupt_on={"get_weather": {"allowed_decisions": ["approve", "reject"]}}),  # HiT-L
        pii_redaction_middleware(patterns=["email", "phone"])  # Scrub PII
    ]
)
```

## Step 5: Custom Middleware (Advanced Hooks)

**Subclass `AgentMiddleware` for `wrap_model_call`, `wrap_tool_call`, etc.**

```python
class DynamicModelMiddleware(AgentMiddleware):
    def wrap_model_call(self, request, handler):
        if len(request.state["messages"]) > 10:  # Complex conv → better model
            request.model = ChatOpenAI(model="gpt-4o")
        return handler(request)

agent = create_agent(model=model, middleware=[DynamicModelMiddleware()])
```

Hooks: `before_model`, `wrap_tool_call`, `after_model`, etc.

## Step 6: Streaming (Real-time UX)

**Use `stream(stream_mode=["updates", "messages", "custom"])` for progress/tokens.**

```python
async for chunk in agent.astream(
    {"messages": [{"role": "user", "content": "Weather?"}]},
    stream_mode=["updates"]  # Or "messages" (tokens), "custom" (tool progress)
):
    print(chunk)  # Step-by-step: model → tools → model
```

**Custom tool streaming:** `config.stream_writer("Fetching weather...")`

## Step 7: Production Features (LangGraph Under the Hood)

**Persistence, HiT-L, time-travel out-of-box:**

```python
from langgraph.store.memory import InMemoryStore  # Or PostgresStore

agent = create_agent(
    model=model,
    store=InMemoryStore(),  # Checkpoints across sessions
    # Interrupts via middleware (Step 4)
)
```

**Debug:** Traces auto-sent to LangSmith.

## [Next: LangGraph for Multi-Agent/Complex Flows]

When `create_agent` limits hit (custom edges, subgraphs), migrate to raw LangGraph graphs.

**Relevant docs:**
- [Agents](https://docs.langchain.com/oss/python/langchain/agents)
- [Tools](https://docs.langchain.com/oss/python/langchain/tools)
- [Middleware](https://docs.langchain.com/oss/python/langchain/middleware/built-in)
- [Streaming](https://docs.langchain.com/oss/python/langchain/streaming)